# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv", parse_dates=['date'])

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

### 1

In [3]:
train['date'].min(), train['date'].max(), test['date'].min(), test['date'].max()

(Timestamp('2016-01-01 00:00:00'),
 Timestamp('2016-08-31 00:00:00'),
 Timestamp('2016-09-10 00:00:00'),
 Timestamp('2016-11-10 00:00:00'))

In [4]:
train_hold = train[train['date'] > pd.to_datetime('2016-06-30')]
train_tr = train[train['date'] <= pd.to_datetime('2016-06-30')]

In [5]:
train_tr['date'].min(), train_tr['date'].max(), train_hold['date'].min(), train_hold['date'].max()

(Timestamp('2016-01-01 00:00:00'),
 Timestamp('2016-06-30 00:00:00'),
 Timestamp('2016-07-01 00:00:00'),
 Timestamp('2016-08-31 00:00:00'))

In [21]:
train_hold['pred_Visits'] = 0.0

In [22]:
train_hold.head()

,Page,date,Visits,pred_Visits
2150330,Special:Search_zh.wikipedia.org_all-access_spider,2016-07-01,765.0,0.0
2150331,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-07-01,6206.0,0.0
2150332,Project:上传_zh.wikipedia.org_all-access_spider,2016-07-01,4253.0,0.0
2150333,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-07-01,382.0,0.0
2150334,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-07-01,16318.0,0.0


### 2

In [3]:
def smape_fast_error(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred.iloc[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [4]:
# kostil = train_tr[train_tr['date'] >= train_tr['date'].max() - relativedelta(days=15)]
unique_pages = train['Page'].unique()

In [69]:
%%time
# i = 0
# st = time.time()
for page in unique_pages:
#     print(kostil[kostil['Page'] == page]['Visits'].median())
    train_hold.loc[train_hold['Page'] == page, 'pred_Visits'] = kostil[kostil['Page'] == page]['Visits'].median()
#     train_hold[train_hold['Page'] == page]['pred_Visits'] = kostil[kostil['Page'] == page]['Visits'].median()
#     if i % 100 == 0:
#         print("Passed", i / len(unique_pages), " time = ", time.time() - st)
#     i += 1

CPU times: user 21min 26s, sys: 24.4 s, total: 21min 51s
Wall time: 21min 57s


In [79]:
smape_fast_error(train_hold['Visits'], train_hold['pred_Visits'])

41.21234998211114

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [5]:
train['Month'] = train["date"].dt.month
train['Day'] = train["date"].dt.day
train['DayOfWeek'] = train["date"].dt.dayofweek

test['Month'] = test["date"].dt.month
test['Day'] = test["date"].dt.day
test['DayOfWeek'] = test["date"].dt.dayofweek

# train_hold = train[train['date'] > pd.to_datetime('2016-06-30')]
# train_tr = train[train['date'] <= pd.to_datetime('2016-06-30')]

def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=2)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=4))].index

In [6]:
train_dates = ['2016-01-01', '2016-03-01', '2016-05-01']
CVIterator = []
for dt in train_dates:
    CVIterator.append(create_validation(train, dt))

In [7]:
hash_pages = {unique_pages[i]:i for i in range(len(unique_pages))}

In [8]:
train['hash_page'] = train['Page'].map(hash_pages)
test['hash_page'] = test['Page'].map(hash_pages)

In [9]:
X = train.drop(['Visits', 'date', 'Page'], axis = 1)
y = train['Visits']

X_test = test.drop(['Visits','date', 'Page'], axis = 1)
y_test = test['Visits']

In [10]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# regressor = SVR()
regressor = RandomForestClassifier()
# regressorLinearRegressionRandomForestRegressor()
#0.01, 10, 100, 1000

# pipeline_r = Pipeline([('regressor', regressor)])
# param_grid = [
#     {'regressor__C': [0.1, 1]}
# #   {'regressor__C': [0.1, 1], 'regressor__kernel': ['linear']},
# #   {'regressor__C': [0.01, 0.1, 1, 10, 100, 1000], 'regressor__gamma': [0.001, 0.0001], 'regressor__kernel': ['rbf']},
#  ]
# grid_search = GridSearchCV(pipeline_r,
#                            param_grid=param_grid,
# #                            cv=CVIterator,
# #                            cv=3,
# #                            scoring=pandas_smape,
#                            n_jobs=-1,
#                            verbose=1)

%time regressor.fit(X,y)

По непонятной причине не хочет (точнее слишком долго) даже обучиться (не говоря про подбор параметров). Возможно неправильно понял как надо работать с страницами. По идее сама страница является предиктором, но не знаю точно как это реализовать...

In [ ]:
X.head()

In [ ]:
grid_search._best_score

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
smape_fast_error(y_test, y_pred)

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [7]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')

In [8]:
import pandas_profiling as pp

In [20]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [21]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


Убираем коррелируюшюю переменную

In [9]:
train.drop('var5', axis=1, inplace=True)
test.drop('var5', axis=1, inplace=True)

Убираем переменные в которых слишком много пропущенных значений

In [10]:
train.drop('sit_juridique', axis=1, inplace=True)
test.drop('sit_juridique', axis=1, inplace=True)

train.drop('var2', axis=1, inplace=True)
test.drop('var2', axis=1, inplace=True)

Остальные пропущенные значения заполняем

In [ ]:
train.fillna(train.mean())
test.fillna(test.mean())

Переводим переменные в числовой вид

In [23]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var1,var3,var4,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,0,500,Indivividu,agent; employe,Sans information,2,34,A,...,4,0,194,0,0,1,NON,30,2,67.68
1,83126,2,0,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,3,0,0,0,0,5,NON,12,1,120.60
2,68224,3,0,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,2,1,98,0,0,5,NON,30,2,100.62
3,30263,3,0,500,Famille,agent; employe,Maison,2,34,A,...,3,0,151,0,0,1,NON,15,1,58.73
4,72231,2,0,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,3,0,39,0,0,1,NON,0,2,132.65


In [15]:
train.shape

(200000, 26)

In [11]:
train['veranda'] = pd.factorize(train['veranda'], sort=True)[0]
train['sit_fam'] = pd.factorize(train['sit_fam'], sort=True)[0]
train['profession'] = pd.factorize(train['profession'], sort=True)[0]
train['nature_lri'] = pd.factorize(train['nature_lri'], sort=True)[0]
train['niv_urbanisation'] = pd.factorize(train['niv_urbanisation'], sort=True)[0]
train['var9'] = pd.factorize(train['var9'], sort=True)[0]

In [29]:
test['veranda'] = pd.factorize(test['veranda'], sort=True)[0]
test['sit_fam'] = pd.factorize(test['sit_fam'], sort=True)[0]
test['profession'] = pd.factorize(test['profession'], sort=True)[0]
test['nature_lri'] = pd.factorize(test['nature_lri'], sort=True)[0]
test['niv_urbanisation'] = pd.factorize(test['niv_urbanisation'], sort=True)[0]
test['var9'] = pd.factorize(test['var9'], sort=True)[0]

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [21]:
param_grid = {
    "n_estimators": [10, 18, 22],
    "max_depth": [3, 5],
    "min_samples_split": [15, 20],
    "min_samples_leaf": [5, 10, 20]
}
regr = RandomForestRegressor()
# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
# clf = GridSearchCV(ensemble.RandomForestRegressor(), tuned_parameters, cv=5, scoring=r2_score, n_jobs=-1, verbose=1)

grid_search = GridSearchCV(regr,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=1)
X = train.drop('prime_tot_ttc', axis=1)
y = train['prime_tot_ttc']
grid_search.fit(X, y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  2.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 18, 22], 'max_depth': [3, 5], 'min_samples_split': [15, 20], 'min_samples_leaf': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [36]:
param_grid = {
    "n_estimators": [10, 18, 22],
    "max_depth": [3, 5],
    "min_samples_split": [15, 20],
    "min_samples_leaf": [5, 10, 20]
}
regr = RandomForestRegressor()
# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
# clf = GridSearchCV(ensemble.RandomForestRegressor(), tuned_parameters, cv=5, scoring=r2_score, n_jobs=-1, verbose=1)

grid_search = GridSearchCV(regr,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring=pandas_smape2,
                           verbose=1)
X = train.drop('prime_tot_ttc', axis=1)
y = train['prime_tot_ttc']
grid_search.fit(X, y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Lib

In [24]:
grid_search.best_score_

0.5919968330926009

In [30]:
X_test = test.drop('prime_tot_ttc', axis=1)
y_test = test['prime_tot_ttc']

In [31]:
test['pred'] = grid_search.best_estimator_.predict(X_test)

In [32]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["prime_tot_ttc"] - df["pred"]) / (df["prime_tot_ttc"] + df["pred"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [35]:
def pandas_smape2(y_true, y_pred):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(y_true - y_pred) / (y_pred + y_true)
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [33]:
pandas_smape(test)

16.395450284763562